In [183]:
import requests 
import json
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import time
import os

In [3]:
def get_bar_data_v5(category, symbol, interval, startTime=None):
    url = "https://api.bybit.com/v5/market/kline"
    if startTime is None:
        req_params = {"category": category, "symbol" : symbol, 'interval' : interval}
    else:
        req_params = {"category": category, "symbol" : symbol, 'interval' : interval, 'start' : startTime}
    df = pd.DataFrame(json.loads(requests.get(url, params = req_params).text)['result'])
    if (len(df.index)==0):
        return None
    df = pd.concat([df['symbol'], pd.DataFrame(list(df['list']), columns=['startTime', 'openPrice', 'highPrice', 'lowPrice', 'closePrice', 'volume', 'turnover'])], axis=1)
    df.index = [dt.datetime.fromtimestamp(int(x)/1000) for x in df.startTime]
    df = df.iloc[::-1]
    return df

In [10]:
# convert date or datetime to seconds, as one argument for v5 API
def dt_to_millsec(date, dateformat='%Y-%m-%d'):
    if len(date)==19 and dateformat=='%Y-%m-%d':
        dateformat = '%Y-%m-%d %H:%M:%S'
    t = dt.datetime.strptime(date, dateformat)
    return int(time.mktime(t.timetuple())*1000)

In [167]:
def get_bar_data_binance(category, symbol, interval, startTime=None):
    url = "https://www.binance.com/fapi/v1/continuousKlines"
    if startTime is None:
        req_params = {"limit": 1500, "pair": symbol, 'contractType': category, 'interval': interval}
    else:
        req_params = {"limit": 1500, "pair": symbol, 'contractType': category, 'startTime': startTime, 'interval': interval}
    df = pd.DataFrame(json.loads(requests.get(url, params = req_params).text)).iloc[:,:-1]
    if (len(df.index)==0):
        return None
    df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'usdt_volume', 'num_trades', 'buy_volume', 'buy_usdt_volume']
    df['symbol'] = symbol
    return df

In [176]:
def query_histdata_binance(category, symbol, interval, sdate=None):
    # interval param: 1m 1h 1d
    if interval == '1d':
        cnt = 86400
    elif interval == '1h':
        cnt = 60
    elif interval == '1m':
        cnt = 1
    else:
        raise ValueError('interval is invalid!')
        return
    if sdate is None:
        sdate = '2020-01-01'
    starttime = dt.datetime.strptime(sdate, '%Y-%m-%d')
    df_list = []
    last_second = dt_to_millsec(sdate)
    while True:
        print(dt.datetime.fromtimestamp((int(last_second/1000))))
        new_df = get_bar_data_binance(category, symbol, interval, last_second)
        if new_df is None or int(new_df['open_time'].max()) < last_second:
            break
        df_list.append(new_df)
        last_second = int(new_df['open_time'].max()) + cnt * 1000
    df = pd.concat(df_list)
    df['open_time'] = [dt.datetime.fromtimestamp(int(x)/1000) for x in df.open_time]
    df['close_time'] = [dt.datetime.fromtimestamp(int(x)/1000) for x in df.close_time]
    return df

In [181]:
def download_crypto_data_binance(symbols, category, saving_folder, interval, sindex=0, tindex=None):
    datadict = {}
    if category == 'futures':
        category = 'PERPETUAL'
    if tindex is None:
        tindex = len(symbols)
    for i in range(sindex, tindex):
        print(symbols[i])
        try:
            tab = query_histdata_binance(category, symbols[i], interval, sdate=None)
            tab.to_csv(os.path.join(saving_folder, symbols[i] + '.csv'))
        except Exception as e:
            print(e)
    return

In [156]:
def get_symbols():
    url = "https://www.binance.com/fapi/v1/exchangeInfo"
    basic_info = pd.DataFrame(json.loads(requests.get(url, params = req_params).text)['symbols'])
    symbols = basic_info[basic_info['contractType']=='PERPETUAL']['symbol'].unique()
    delist_pairs = ['1000BTTCUSDT', 'YFIIUSDT', 'FTTBUSD', 'LUNAUSDT', 'ANCUSDT', 'AKROUSDT', 'DODOUSDT', 'TLMUSDT', 'SCUSDT', 'BTSUSDT', 'BLZUSDT', 'RAYUSDT', 'SRMUSDT', 'CVCUSDT', 'MINAUSDT',
                'CVXBUSD', 'GALBUSD', 'WAVESBUSD', 'ICPBUSD', 'LUNA2BUSD', 'AMBBUSD', 'LEVERBUSD', 'TLMBUSD', 'PHBBUSD', 'SANDBUSD', 'UNIBUSD', 'GMTBUSD', 'NEARBUSD', 'AVAXBUSD', 'ETCBUSD',
                'FILBUSD', 'DOTBUSD', 'APEBUSD', 'LINKBUSD', '1000SHIBBUSD', '1000LUNCBUSD']
    symbols = list(set(symbols).union(set(delist_pairs)))
    symbols = [symbol for symbol in symbols if symbol[-4:]=='USDT']
    symbols.sort()
    return symbols

In [157]:
symbols = get_symbols()

In [184]:
download_crypto_data_binance(symbols, 
                             'PERPETUAL', 
                             r'C:\Users\Administrator\Documents\jupyter_notebook\data\binance', 
                             '1d')

1000BTTCUSDT
2020-01-01 00:00:00
2022-04-19 08:00:00
1000FLOKIUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
1000LUNCUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
1000PEPEUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
1000SHIBUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
1000XECUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
1INCHUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
AAVEUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
ACHUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
ADAUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
AGIXUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
AKROUSDT
2020-01-01 00:00:00
2022-10-14 08:00:00
ALGOUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
ALICEUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
ALPHAUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
AMBUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
ANCUSDT
2020-01-01 00:00:00
2022-05-25 08:00:00
ANKRUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
ANTUSDT
2020-01-01 00:00:00
2023-06-08 08:00:00
HTTPSConnectionPool(host='www.binance.com', por

KeyboardInterrupt: 